In [1]:
%env CUDA_VISIBLE_DEVICES=5

import torch
from torch import nn

from linear import HiggsLinear

def replace_with_higgs_linear(
    model,
    quantization_config=None,
    current_key_name=None,
    has_been_replaced=False,
):
    for name, module in model.named_children():
        if current_key_name is None:
            current_key_name = []
        current_key_name.append(name)

        if isinstance(module, nn.Linear):
            # Check if the current key is not in the `linear_weights_not_to_quantize`
            if ".".join(current_key_name) in quantization_config:
                in_features = module.in_features
                out_features = module.out_features
                higgs_d = quantization_config[".".join(current_key_name)]

                model._modules[name] = HiggsLinear(
                    in_features,
                    out_features,
                    higgs_d,
                    bias=module.bias is not None,
                    dtype=module.weight.dtype,
                )
                has_been_replaced = True

                # Store the module class in case we need to transpose the weight later
                model._modules[name].source_cls = type(module)
                # Force requires grad to False to avoid unexpected errors
                model._modules[name].requires_grad_(False)
        if len(list(module.children())) > 0:
            _, has_been_replaced = replace_with_higgs_linear(
                module,
                quantization_config=quantization_config,
                current_key_name=current_key_name,
                has_been_replaced=has_been_replaced,
            )
        # Remove the last key for recursion
        current_key_name.pop(-1)
    return model, has_been_replaced

env: CUDA_VISIBLE_DEVICES=5


/nfs/scistore19/alistgrp/apanfero/GPTAQ/.conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/nfs/scistore19/alistgrp/apanfero/linear-layer-compression/Andrei/kernels/higgs.py:26: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("higgs::higgs_matmat")


In [2]:
from transformers import AutoModelForCausalLM
from accelerate import init_empty_weights


model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B", torch_dtype=torch.float16, attn_implementation="eager")

original_weight = model.model.layers[0].self_attn.q_proj.weight.data.clone().detach()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
from typing import Optional

def build_layerwise_edenn_config(
    edenn_d: Optional[int] = None, 
    blockwise_edenn_config: Optional[list[int]] = None,
    layerwise_edenn_config: Optional[dict[str, int]] = None,
) -> list[(int, int)]:
    if layerwise_edenn_config is not None:
        assert edenn_d is None and blockwise_edenn_config is None
        return layerwise_edenn_config
    
    if blockwise_edenn_config is None:
        assert edenn_d is not None
        blockwise_edenn_config = [edenn_d for _ in range(32)]
    
    layer_names = [
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj",
    ]
    
    return {
        f"model.layers.{i}.{layer_name}": blockwise_edenn_config[i]
        for layer_name in layer_names
        for i in range(len(blockwise_edenn_config))
    }


In [4]:
model, _ = replace_with_higgs_linear(
    model,
    build_layerwise_edenn_config(1),
)

In [5]:
from accelerate import load_checkpoint_and_dispatch

model = load_checkpoint_and_dispatch(
    model,
    "/nfs/scistore19/alistgrp/apanfero/models/higgs/Meta-Llama-3.1-8B.pt",
).to("cuda")

/nfs/scistore19/alistgrp/apanfero/GPTAQ/.conda/lib/python3.10/site-packages/accelerate/utils/modeling.py:1466: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.loa

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")

In [7]:
tokenizer.decode(
    model.generate(**tokenizer("Hi!", return_tensors='pt').to("cuda"))[0].cpu()
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


/nfs/scistore19/alistgrp/apanfero/GPTAQ/.conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


"<|begin_of_text|>Hi! I'm new to this forum. I'm a 19 year old female who has"